In [83]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import Image
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    driver.quit()
except:
    print('starting fresh')
    
#driver = webdriver.Chrome('../../hackStacks/chromedriver')
driver = webdriver.PhantomJS()
driver.get('https://wild949.iheart.com/music/recently-played/')

In [84]:
# wait = WebDriverWait(driver, 10)
# try:
#     for i in range(10):
#         element = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn-lg load-more station-custom-button']")))
#         driver.find_element_by_xpath("//button[@class='btn btn-lg load-more station-custom-button']").click()
#         if i % 5 == 0:
#             print(i)
# except:
#     # troubleshooting
#     print('we failed somewhere :(')
#     driver.save_screenshot('out.png')
#     Image(filename='out.png') 

In [85]:
%%time
for i in range(92):
    driver.find_element_by_xpath("//button[@class='btn btn-lg load-more station-custom-button']").click()
    time.sleep(5)
    if i % 3 == 0:
        print(i, 'clicks')
    
soup = BeautifulSoup(driver.page_source)

driver.quit()

0 clicks
3 clicks
6 clicks
9 clicks
12 clicks
15 clicks
18 clicks
21 clicks
24 clicks
27 clicks
30 clicks
33 clicks
36 clicks
39 clicks
42 clicks
45 clicks
48 clicks
51 clicks
54 clicks
57 clicks
60 clicks
63 clicks
66 clicks
69 clicks
72 clicks
75 clicks
78 clicks
81 clicks
84 clicks
87 clicks
90 clicks


/Users/ronaldwalker/anaconda/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


CPU times: user 5.21 s, sys: 1.18 s, total: 6.4 s
Wall time: 7min 55s


In [60]:
# troubleshooting
# driver.save_screenshot('out.png')
# Image(filename='out.png') 

In [86]:
songs = soup.findAll('li', {'class': 'playlist-track-container'})

titles = []
artists = []
times = []

for song in songs:
    song_data = [x for x in song.text.split('\n') if x != '']
    titles.append(song_data[0])
    artists.append(song_data[1])
    times.append(song_data[4])

In [87]:
first_date = soup.find('li', {'class': 'playlist-date-header'})

In [88]:
print(pd.Series(titles).value_counts()[:10])

Feel It Still                 48
Havana                        42
rockstar [Feat. 21 Savage]    42
What Lovers Do                40
Thunder                       38
Perfect                       31
How Long                      30
Bad At Love                   27
New Rules                     25
Attention                     23
dtype: int64


In [89]:
data = pd.DataFrame({'title': titles, 'artist': artists, 'time': times})
data.head()

,artist,time,title
0,Post Malone,10:59 AM,rockstar [Feat. 21 Savage]
1,Portugal. The Man,10:53 AM,Feel It Still
2,French Montana feat. Swae Lee,10:50 AM,Unforgettable
3,Charlie Puth,10:46 AM,How Long
4,Niall Horan,10:36 AM,Slow Hands


In [90]:
print('number of songs', len(data))

number of songs 900


In [91]:
actual_start_date = int([x.split('\n') for x in first_date.text.split(' ')][2][0])

In [92]:
day_number = actual_start_date
month_number = 12
last = 'PM'
days = []
months = []
for t in data['time']:
    tod = t[-2:]
    if last == 'AM' and tod == 'PM':
        day_number -= 1
    if day_number == 0:
        day_number = 30
        month_number -= 1
    days.append(day_number)
    months.append(month_number)
    last = tod

In [93]:
data['day'] = days
data['month'] = months
data['station'] = '94.9'

In [94]:
data.head()

,artist,time,title,day,month,station
0,Post Malone,10:59 AM,rockstar [Feat. 21 Savage],2,12,94.9
1,Portugal. The Man,10:53 AM,Feel It Still,2,12,94.9
2,French Montana feat. Swae Lee,10:50 AM,Unforgettable,2,12,94.9
3,Charlie Puth,10:46 AM,How Long,2,12,94.9
4,Niall Horan,10:36 AM,Slow Hands,2,12,94.9


In [96]:
data.query('time == "3:52 PM"')

,artist,time,title,day,month,station
486,Charlie Puth,3:52 PM,How Long,30,11,94.9


In [69]:
data.to_csv(, index=False)